In [1]:
import tensorflow as tf
import os
import data_loader
import model
import main_module
import numpy as np


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
data = data_loader.read_datasets()

------------------read datasets begin-------------------
------------------load word2vec begin-------------------


------------------load word2vec end---------------------


------------------read datasets end---------------------


In [3]:
# load settings
vocab_size, word_emb_size = data['embedding'].shape
_, max_sentence_length = data['x_tr'].shape
intents_number = len(data['intents_dict'])
slots_number = len(data['slots_dict'])

FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_float("keep_prob", 0.8, "embedding dropout keep rate")
tf.app.flags.DEFINE_integer("hidden_size", 32, "embedding vector size")
tf.app.flags.DEFINE_integer("batch_size", 1, "vocab size of word vectors")
tf.app.flags.DEFINE_integer("num_epochs", 20, "num of epochs")
tf.app.flags.DEFINE_integer("vocab_size", vocab_size, "vocab size of word vectors")
tf.app.flags.DEFINE_integer("max_sentence_length", max_sentence_length, "max number of words in one sentence")
tf.app.flags.DEFINE_integer("intents_nr", intents_number, "intents_number")  #
tf.app.flags.DEFINE_integer("slots_nr", slots_number, "slots_number")  #
tf.app.flags.DEFINE_integer("word_emb_size", word_emb_size, "embedding size of word vectors")
tf.app.flags.DEFINE_boolean("use_embedding", True, "whether to use embedding or not.")
tf.app.flags.DEFINE_float("learning_rate", 0.01, "learning rate")
tf.app.flags.DEFINE_float("margin", 1.0, "ranking loss margin")
tf.app.flags.DEFINE_integer("slot_routing_num", 4, "slot routing num")
tf.app.flags.DEFINE_integer("intent_routing_num", 4, "intent routing num")
tf.app.flags.DEFINE_integer("re_routing_num", 3, "re routing num")
tf.app.flags.DEFINE_integer("intent_output_dim", 64, "intent output dimension")
tf.app.flags.DEFINE_integer("slot_output_dim", 128, "slot output dimension")
tf.app.flags.DEFINE_integer("attention_output_dimenison", 20, "self attention weight hidden units number")
tf.app.flags.DEFINE_float("alpha", 0.001, "coefficient for self attention loss")
tf.app.flags.DEFINE_integer("r", 3, "self attention weight hops")
tf.app.flags.DEFINE_boolean("save_model", False, "save model to disk")
tf.app.flags.DEFINE_boolean("test", False, "Evaluate model on test data")
tf.app.flags.DEFINE_boolean("crossval", False, "Perform k-fold cross validation")
tf.app.flags.DEFINE_integer("n_splits", 3, "Number of cross-validation splits")
tf.app.flags.DEFINE_string("summaries_dir", './logs', "tensorboard summaries")
tf.app.flags.DEFINE_string("ckpt_dir", './saved_models/Scenario0', "check point dir")
tf.app.flags.DEFINE_string("scenario_num", '', "Scenario number")
tf.app.flags.DEFINE_string('f', '', 'kernel')  # required by jupyter notebook 

In [7]:
tf.reset_default_graph()
config = tf.ConfigProto()
sess = tf.Session(config=config) 
# Instantiate Model
capsnet = model.capsnet(FLAGS)
if os.path.exists(FLAGS.ckpt_dir):
    print("Restoring Variables from Checkpoint for testing")
    saver = tf.train.Saver()
    saver.restore(sess, tf.train.latest_checkpoint(FLAGS.ckpt_dir))
else:
    print("No trained model exists in checkpoint dir!")

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API


Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


Instructions for updating:
Use tf.cast instead.


...

Instructions for updating:
dim is deprecated, use axis instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


Restoring Variables from Checkpoint for testing
Instructions for updating:
Use standard file APIs to check for files with this prefix.


INFO:tensorflow:Restoring parameters from ./saved_models/Scenario0\model.ckpt-1


Utterance classified as: scadeIntensitateLumina
Slots:
['O', 'O', 'O', 'O', 'O', 'O', 'B-loc', 'O', 'O', 'O', 'O', 'O', 'O', 'B-loc', 'O']
Finished inferring
['O', 'O', 'O', 'O', 'O', 'O', 'B-loc', 'O', 'O', 'O', 'O', 'O', 'O', 'B-loc', 'O']
2


In [4]:
input_sentence = 'fa mai mica intensitatea iluminatului in sufragerie la 4'

fa mai mica intensitatea iluminatului in sufragerie la 4


In [5]:
sentence_words = input_sentence.split(' ')
encoded_words = []
for w in sentence_words:
    encoded_words.append(data_loader.w2v_dict['w2v'].vocab[w].index)
    
print(encoded_words)

fa mai mica intensitatea iluminatului in sufragerie la 4
ba
[11165, 23, 5076, 10840, 66383, 50, 30878, 8, 129]


In [6]:
max_len = data['max_len']
print("original sentencce length" + len(encoded_words))
encoded_padded_words = np.append(encoded_words, np.zeros((max_len - len(encoded_words),), dtype=np.int64))
data['sample_utterance'] = encoded_padded_words
print("padded sentence length" + len(encoded_padded_words))

9


In [ ]:
slots, intents = main_module.evaluate_sample(capsnet, data, sess)
